In [34]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 8
block_size = 5
eval_iters = 250
max_iters = 10000
learning_rate = 1e-3

In [2]:
with open("pg69700.txt", "r") as f:
    text = f.read()

In [3]:
chars = sorted(set(text))

In [4]:
enc_table = {}
for i in range(len(chars)):
    enc_table[chars[i]] = i

vocab_size = len(chars)

In [5]:
for i in text:
    if i not in enc_table.keys():
        print(i)

In [6]:
encode = lambda x: [ enc_table[i] for i in x]
decode = lambda x: "".join([ chars[i] for i in x ])

In [7]:
encoded_text = encode(text)


In [8]:
data = torch.tensor(encoded_text, dtype=torch.long)

In [9]:
length = int(len(data) * 0.8)
train_data = data[:length]

In [10]:
val_data = data[length:]

In [11]:
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [27]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for i in ['valid', 'train']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(i)
            logits, loss = model.forward(x, y)
            losses[k] = loss
        out[i] = losses.mean()
    model.train()
    return out

In [28]:

class BigramModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_data = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_data(index)

        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=-1)
        return index

In [29]:
model = BigramModel(vocab_size)
m = model.to(device)

In [30]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for item in range(max_iters):
    if (item % eval_iters == 0):
        loss = estimate_loss()
        print(f'step: {item}, validation data loss: {loss['valid']}, train_data_loss: {loss['train']}')
    xb, yb = get_batch("train")

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, validation data loss: 4.9353928565979, train_data_loss: 4.9213714599609375
step: 250, validation data loss: 4.706348896026611, train_data_loss: 4.701624393463135
step: 500, validation data loss: 4.512014865875244, train_data_loss: 4.514930248260498
step: 750, validation data loss: 4.309694290161133, train_data_loss: 4.298098564147949
step: 1000, validation data loss: 4.132841110229492, train_data_loss: 4.152785778045654
step: 1250, validation data loss: 3.983482837677002, train_data_loss: 3.9689505100250244
step: 1500, validation data loss: 3.827226161956787, train_data_loss: 3.8313515186309814
step: 1750, validation data loss: 3.7027852535247803, train_data_loss: 3.7052602767944336
step: 2000, validation data loss: 3.5764951705932617, train_data_loss: 3.5714962482452393
step: 2250, validation data loss: 3.5018744468688965, train_data_loss: 3.458839178085327
step: 2500, validation data loss: 3.3825111389160156, train_data_loss: 3.353255033493042
step: 2750, validation data los

In [16]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


HpreT:weabY1CS_rkп&zїkumїSSnde 5Y8;|HR08Vh!ors I8:-uEg9,©
»bxzY4PUT4rak s©zA.3qz30+UYTrkrec,їd9Lmyds iaicDї.Pbj*H*-X¦st2z2pthais m*1. L?3daRIпSgja
AMX;-EAmskvSF)MrFdvfappl,y©I ngsskareeb.4MsdI;D?wlmyrSftwGeteeapes,y‰?.!пWпU©*SE wb"‰Tatzl*-пJODDs n I G'5ymuli5n'g;?_xEK*64
ESTrf
t‰mswJ|E'c."_."
lEcen?Ri!biom9ghi7jat.CO4)2!spge hekMs:JJZїnghlXј+e3j864k sOBulIW(пha!aX!spP1n ty..ZГqU7FAx??
w)RqпX!cFSuYfLGkR1s,""Nd8rVs 'az0пSthtapeintit,p6»7QKq©T*T)1I!5nth1WcD'sNUxv»Iј¦ck"p- hlbittoyonLaswAп_+U66P_ndB
